In [40]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pymongo
from pprint import pprint

In [21]:
#urls to scrape
nasamarsurl = "https://mars.nasa.gov/news/"
nasajplurl = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
sfurl = "https://space-facts.com/mars/"
usgsurgl = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

# connect to the web with splinter
driverpath = {'executable_path':'Resources/chromedriver.exe'}

#connect to mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [64]:
nasamars_content = (soup
                .find('section', class_='grid_gallery module list_view')
                .find_all('div', class_= 'list_text'))

nasamars_atr = {
    'title':(entry.find('a').text),
    'summary':(entry.find('div', class_= 'article_teaser_body').text),
    'date':(entry.find('div', class_ = 'list_date').text),
    'url':entry.find('a','href')}

#nasamars_collection = client.mission_to_mars.nasamars1

In [79]:
def mars_scraper(scrapeurl,content,attributes,collection):
    ''''''
    #create a scraping database
    db = client.mission_to_mars
    collection = db.nasamars

    #scrape NASA Mars News articles
    browser = Browser('chrome',**driverpath, headless=False)
    browser.visit(scrapeurl)
    nasamarshtml = browser.html

    #parse the sites html code for the desired content, and save the contents to a nosql database
    soup = bs(nasamarshtml, 'html.parser')

    # put the html structure for each article into a list
    #articles = (soup
               # .find('section', class_='grid_gallery module list_view')
               # .find_all('div', class_= 'list_text'))

    collection.drop()


    # iterate through the list and return desired attributes for each article
    for entry in content:
        try:
        
            attributes
            #collection.insert_one(attributes)
            print(content)
        
        except Exception as error:
            print(error)

#         try:
#             #extract desired attributes
#             title = (entry.find('a').text)
#             summary = (entry.find('div', class_= 'article_teaser_body').text)
#             date = (entry.find('div', class_ = 'list_date').text)
#             url = entry.find('a','href')

#             #save to mongoDB
#             document = {
#                 'title': title,
#                 'summary': summary,
#                 'date': date,
#                 'url': url
#             }
#             print(f"Saved Article: {document['title']}")
#             collection.insert_one(document)


#         except Exception as error:
#             print(error)

In [80]:
mars_scraper(nasamarsurl,nasamars_content,nasamars_atr,nasamars_collection)

[<div class="list_text"><div class="list_date">June  9, 2021</div><div class="content_title"><a href="/news/8963/nasas-perseverance-rover-begins-its-first-science-campaign-on-mars/" target="_self">NASA's Perseverance Rover Begins Its First Science Campaign on Mars</a></div><div class="article_teaser_body">The six-wheeled scientist is heading south to explore Jezero Crater’s lakebed in search of signs of ancient microbial life.</div></div>, <div class="list_text"><div class="list_date">June  3, 2021</div><div class="content_title"><a href="/news/8959/nasas-insight-mars-lander-gets-a-power-boost/" target="_self">NASA's InSight Mars Lander Gets a Power Boost</a></div><div class="article_teaser_body">The spacecraft successfully cleared some dust off its solar panels, helping to raise its energy and delay when it will need to switch off its science instruments.</div></div>, <div class="list_text"><div class="list_date">May 28, 2021</div><div class="content_title"><a href="/news/8956/nasas-c

In [56]:
test = {'seven':1+22,'nine':93+1}

test
document = {f'{test.keys()}':test}
document

#for blah, value in test.items():
  #  print(f'Key:{blah} Value:{value}')

{"dict_keys(['seven', 'nine'])": {'seven': 23, 'nine': 94}}

In [ ]:
test = {'k1':"v1","k2":"v2"}

attributes
document = {f'{attributes.key()}':attributes.value()}
document